## **Trabajo Práctico Individual n° 2: Aprendizaje Supervisado**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

- Nombre:clara
- Apellido: schiebelbein
- Legajo: 1526005

En este trabajo práctico aplicaremos algunas técnicas aprendidas para el aprendizaje automático supervisado.

Utilizaremos un dataset de airbnb donde buscaremos predecir el precio de las noches de cada hogar en funcion de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

In [1]:
# Verificamos si estamos en Colab
var_google_colab = 'google.colab' in str(get_ipython())
print(var_google_colab)
if var_google_colab: 
  #Montamos nuestro G.Drive
  from google.colab import drive
  drive.mount('/content/gdrive',force_remount=True)

True
Mounted at /content/gdrive


In [3]:
 if var_google_colab:
 # Direccion root donde está la jupyter-notebook
  root_path = "/content/gdrive/MyDrive/UTN-FRBA/Ciencia de datos/TP/"


## **Objetivos del trabajo practico**
- resolver los requisitos que se detallan en las proximas celdas
- dejar comentarios # en cada linea de codigo que se agregue
- fecha limite de entrega: 13 de Octubre 2022 a las 23h59

In [4]:
## Importo librerías de maniupulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
## Importo librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.svm import SVC

##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [6]:
# importo el dataset london
london= pd.read_csv(root_path+'TP2_airbnb_london.csv')
#Vemos que se haya importado correctamente
london.head()

,Unnamed: 0,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0,Haringey,Private room,35,1,131,1.71,4,262
1,1,Ealing,Private room,70,2,2,0.07,6,62
2,2,Islington,Private room,45,1,14,0.14,2,364
3,5,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,6,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london" 

In [7]:
#imprimo las dimensiones del dataset. 
london.shape

(51827, 9)

In [8]:
#elimino la columna Unnamed:0
london = london.drop(['Unnamed: 0'], axis=1)

In [9]:
#muestro las primeras lineas del dataset london
london.head()

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Haringey,Private room,35,1,131,1.71,4,262
1,Ealing,Private room,70,2,2,0.07,6,62
2,Islington,Private room,45,1,14,0.14,2,364
3,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [10]:
#verifico que el dataset no cuenta con valores nulos
london.isnull().any()

neighbourhood                     False
room_type                         False
price                             False
minimum_nights                    False
number_of_reviews                 False
reviews_per_month                 False
calculated_host_listings_count    False
availability_365                  False
dtype: bool

##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [11]:
#muestro el tipo de dato  neighbourhood
london['neighbourhood'].dtype


dtype('O')

In [12]:
#muestro el tipo de dato room_type
london['room_type'].dtype

dtype('O')

In [13]:
#Un tipo ‘O’ solo significa “objeto” que en el mundo de Pandas es una secuencia de caracteres (texto).

##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset london.


In [14]:
# Genero las dummies para la variable neighbourhood, dentro de la variable neigh_dummies.
neigh_dummies = pd.get_dummies(london.neighbourhood)

Imprimir las dimensiones y primeras lineas de estas dummies

In [15]:
#imprimo la dimension y las primeras lineas de las dummnies
neigh_dummies.shape

(51827, 33)

In [16]:
#muestro la variable
neigh_dummies.head()

,Barking and Dagenham,Barnet,Bexley,Brent,Bromley,Camden,City of London,Croydon,Ealing,Enfield,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Genero las dummies para la variable room_type, dentro de la variable room_dummies.
room_dummies =pd.get_dummies(london.room_type)

Imprimir las dimensiones y primeras lineas de estas dummies

In [18]:

#imprimo la dimension y muestro las primeras lineas
room_dummies.shape

(51827, 3)

In [19]:
room_dummies.head()

,Entire home/apt,Private room,Shared room
0,0,1,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


Joinear las variables dummies al dataset



In [20]:
#agrego al dataset London , las nuevas variables dummnies
london = london.join(room_dummies)

In [21]:
london = london.join(neigh_dummies)

In [22]:
london.head()

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,Entire home/apt,Private room,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,Haringey,Private room,35,1,131,1.71,4,262,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Ealing,Private room,70,2,2,0.07,6,62,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Islington,Private room,45,1,14,0.14,2,364,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295,1,0,...,0,0,0,0,0,0,1,0,0,0
4,Barnet,Private room,29,5,114,1.13,3,285,0,1,...,0,0,0,0,0,0,0,0,0,0


##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [23]:
#genero las variables y e x son valores del dataset
y = np.array(london[["price"]])
x = london.drop([ 'price','neighbourhood','room_type'], axis=1)

##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [24]:
# obtengo xtrain, xtest, ytr e ytest  tomando 30%
xtrain, xte, ytr, yte = train_test_split(x, y, test_size=0.3, random_state=4)

In [25]:
# muestro la dimension xtrain
xtrain.shape


(36278, 41)

In [26]:
# veo como queda la matriz de xte
xte.shape

(15549, 41)

##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


In [27]:
scaler = preprocessing.StandardScaler().fit(xtrain)
# declaro un standard scaler y lo ajusto a los datos de entrenamiento
scaler

StandardScaler()

In [28]:
# xtrain_scal tendra el dataset de train pre-procesado con el standard scaler
xtrain_scal = scaler.transform(xtrain)

In [29]:
# auto scalo mis muestras de test utilizando el scaler fiteado con el xtrain
xtest_scal = scaler.transform(xte)


In [30]:
xtrain_scal.shape


(36278, 41)

In [31]:
xtest_scal.shape

(15549, 41)

##### **Objetivo 9**
Crear un modelo lineal y entrenarlo para nuestro dataset.

Obtener las predicciones y el "mean_squared_error" para el test set.

In [32]:
# creo un modelo de regresion lineal
lr = LinearRegression()

In [33]:
#  Ajusto el modelo de regresion lineal utilizando el set de train, tanto las features X como las samples Y
lr.fit(xtrain_scal, ytr)

LinearRegression()

In [34]:
# Obtengo las predicciones que realiza mi modelo con las muestras de test, sin mostrarle las labels (Ytest)
# las predicciones las guardo en el vector "ypred"
ypred = lr.predict(xtest_scal)

In [35]:
np.shape(ypred)


(15549, 1)

In [36]:
# calculo el error de mi modelo con las muestras de train = error de train
np.sqrt(mean_squared_error(yte, ypred))


39.33143615572913

##### **Objetivo 10**
Crear un SVM Regressor y un GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [37]:
# importo gridsearchCV
from sklearn.model_selection import GridSearchCV
#defino los parametros
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10, 100], 'gamma':[0.1,0.5,1]}


In [38]:
# Ahora cambiamos el estimador, usaremos SVR
#aclaracion: max_iter fue para determinar la cantidad de iteracion, porque el tiempo de ejecucion era muy largo
svreg = SVR(max_iter=1000)
# Defino nuevamente el objeto con los parametros previamente definidos
clf_svm = GridSearchCV(svreg, param_grid = parameters, refit = True, cv = 5, verbose= 1)

In [ ]:

# Ajusto mi modelo a las muestras de training utilizando tanto Xtrain_cal como Ytr
#clf_svm.fit(xtrain_scal, ytr.ravel()) 

clf_svm.fit (xtrain_scal, ytr)

##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set.

In [40]:
#imprimo en pantalla los mejores hiperparamentros
print(clf_svm.best_estimator_, "\n")
print(clf_svm.best_params_, "\n")
print(clf_svm.best_score_, "\n")


SVR(C=1, gamma=0.1, max_iter=1000) 

{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'} 

-0.1948651925596106 



In [41]:
ypred3 = clf_svm.predict(xtest_scal)

In [42]:
# calculo el error de mi modelo con las muestras 
np.sqrt(mean_squared_error(yte, ypred3))

62.88295590810538

##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.

In [53]:
#defino nuevos parametros basandome en el resultado de lo obtenido en el punto 11
parameters1 = {'kernel':('linear', 'rbf'), 'C':[0.01, 0.1,1], 'gamma':[0.001,0.01,0.1]}


In [54]:
svreg1 = SVR(max_iter=1000)
# Definimos nuevamente el objeto GS con los parametros previamente definidos
clf_svm1 = GridSearchCV(svreg1, param_grid = parameters1, refit = True, cv = 5, verbose= 1)

In [ ]:
# Ajusto mi modelo a las muestras de training utilizando tanto Xtrain como Yt
clf_svm1.fit(xtrain_scal, ytr.ravel()) 

In [56]:
#imprimo en pantalla los mejores hiperparamentros
print(clf_svm1.best_estimator_, "\n")
print(clf_svm1.best_params_, "\n")
print(clf_svm1.best_score_, "\n")


SVR(C=0.1, gamma=0.001, kernel='linear', max_iter=1000) 

{'C': 0.1, 'gamma': 0.001, 'kernel': 'linear'} 

-0.08814814138774847 



In [ ]:
#por los valores se puede observar que el modelo mejoro

In [57]:
ypred4 = clf_svm1.predict(xtest_scal)

In [58]:
# calculo el error de mi modelo con las muestras 
np.sqrt(mean_squared_error(yte, ypred4))

60.772035507566464